### Listen to the merger converted to audio ###

In [1]:
import sys
!{sys.executable} -m pip install pycbc --no-cache-dir

In [2]:
#Set up the libraries we need and download the data
!wget https://losc.ligo.org/s/events/GW150914/H-H1_LOSC_4_V2-1126259446-32.gwf

--2020-06-27 14:23:29--  https://losc.ligo.org/s/events/GW150914/H-H1_LOSC_4_V2-1126259446-32.gwf
Resolving losc.ligo.org (losc.ligo.org)... 131.215.113.73
Connecting to losc.ligo.org (losc.ligo.org)|131.215.113.73|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gw-openscience.org/s/events/GW150914/H-H1_LOSC_4_V2-1126259446-32.gwf [following]
--2020-06-27 14:23:30--  https://www.gw-openscience.org/s/events/GW150914/H-H1_LOSC_4_V2-1126259446-32.gwf
Resolving www.gw-openscience.org (www.gw-openscience.org)... 131.215.113.73
Connecting to www.gw-openscience.org (www.gw-openscience.org)|131.215.113.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029288 (1005K)
Saving to: ‘H-H1_LOSC_4_V2-1126259446-32.gwf’

H-H1_LOSC_4_V2-1126 100%[===================>]   1005K   613KB/s    in 1.6s    

2020-06-27 14:23:33 (613 KB/s) - ‘H-H1_LOSC_4_V2-1126259446-32.gwf’ saved [1029288/1029288]



In [3]:
from pycbc.frame import read_frame
from pycbc.filter import highpass_fir, lowpass_fir
from pycbc.psd import welch, interpolate
from pycbc.types import TimeSeries

# Read data and remove low frequency content
fname = 'H-H1_LOSC_4_V2-1126259446-32.gwf'
h1 = highpass_fir(read_frame(fname, 'H1:LOSC-STRAIN'), 15.0, 8)

# estimate the noise spectrum and whiten
psd = interpolate(welch(h1), 1.0 / 32)
white_strain = (h1.to_frequencyseries() / psd ** 0.5 * psd.delta_f).to_timeseries()

# remove some of the high and low frequencies
smooth = highpass_fir(white_strain, 25, 8)
smooth = lowpass_fir(white_strain, 250, 8)

# slow the data down by a factor of 4
smooth = TimeSeries(smooth, delta_t=smooth.delta_t*4)

#strech out and shift the frequency upwards by 300 Hz to aid human hearing
frequency_shift = 300.0
fdata = smooth.to_frequencyseries()
fdata.roll(int(frequency_shift / fdata.delta_f))
smooth = fdata.to_timeseries()

#Take slice around signal
smooth = smooth[len(smooth)//2 - 1500:len(smooth)//2 + 3000]
smooth.save_to_wav('gw150914_h1_chirp.wav')

error: required argument is not an integer

### Downlod your audio file ###
 1. If using JupyterLab, your file should show up in the sidebar
 2. Select and download gw150914_h1_chirp.wav